# Impory libraries

In [47]:
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import calendar
from numpy import asarray
from category_encoders import OrdinalEncoder
import gc
import lightgbm as lgbm
import warnings
import numpy as np
import sklearn

In [ ]:
pd.set_option('display.max_rows', 400)
pd.set_option('display.max_columns', 160)
pd.set_option('display.max_colwidth', 40)
warnings.filterwarnings("ignore")

# Task: Sales forecasting for Day-Store-Item

# Metrics for model evaluation

In [45]:
def rmse(y_true, y_pred):

    return np.sqrt(
        sklearn.metrics.mean_squared_error(
            y_true=y_true,
            y_pred=y_pred
        )
    )

# Dataset description

In [4]:
df_train = pd.read_csv(r'train.csv')
df_test = pd.read_csv(r'test.csv')
df_submission = pd.read_csv(r'submission.csv')
df_store = pd.read_csv(r'store.csv')
df_article = pd.read_csv(r'article.csv')
df_sales_count = pd.read_csv(r'sales_count.csv')
df_holidays = pd.read_csv(r'holidays.csv')

df_train['DAY_ID'] = pd.to_datetime(df_train['DAY_ID'])
df_test['DAY_ID'] = pd.to_datetime(df_test['DAY_ID'])
df_submission['DAY_ID'] = pd.to_datetime(df_submission['DAY_ID'])
df_sales_count['DAY_ID'] = pd.to_datetime(df_sales_count['DAY_ID'])
df_holidays['DAY_ID'] = pd.to_datetime(df_holidays['DAY_ID'])

In [5]:
df_train['DAY_ID'] = pd.to_datetime(df_train['DAY_ID'])
print(df_train.shape)
df_train.head()

(6448195, 5)


,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION
0,2017-06-01,1,99197,3.0,0
1,2017-06-01,1,103520,2.0,0
2,2017-06-01,1,105574,11.0,0
3,2017-06-01,1,105575,9.0,0
4,2017-06-01,1,105577,2.0,0


In [6]:
df_test.head()

,DAY_ID,STORE_ID,ARTICLE_ID,IS_ACTION
0,2017-08-01,1,103520,0
1,2017-08-01,1,103665,0
2,2017-08-01,1,105574,0
3,2017-08-01,1,105575,0
4,2017-08-01,1,105693,0


In [7]:
print('train')
print('минимальная дата', df_train['DAY_ID'].min())
print('максимальная дата', df_train['DAY_ID'].max())

print('\ntest')
print('минимальная дата', df_test['DAY_ID'].min())
print('максимальная дата', df_test['DAY_ID'].max())

print('\nsubmission')
print('минимальная дата', df_submission['DAY_ID'].min())
print('максимальная дата', df_submission['DAY_ID'].max())

print('\nsales_count')
print('минимальная дата', df_sales_count['DAY_ID'].min())
print('максимальная дата', df_sales_count['DAY_ID'].max())

print('\nholidays')
print('минимальная дата', df_holidays['DAY_ID'].min())
print('максимальная дата', df_holidays['DAY_ID'].max())

train
минимальная дата 2017-06-01 00:00:00
максимальная дата 2017-07-31 00:00:00

test
минимальная дата 2017-08-01 00:00:00
максимальная дата 2017-08-15 00:00:00

submission
минимальная дата 2017-08-01 00:00:00
максимальная дата 2017-08-15 00:00:00

sales_count
минимальная дата 2017-06-01 00:00:00
максимальная дата 2017-08-15 00:00:00

holidays
минимальная дата 2017-04-14 00:00:00
максимальная дата 2017-12-25 00:00:00


## Sales and returns in total

In [8]:
df_train['SALES'].apply(lambda x: True if x>0 else False).value_counts()

True     6447711
False        484
Name: SALES, dtype: int64

## Number of records for each store

In [9]:
temp = pd.DataFrame(df_train['STORE_ID'].value_counts()).reset_index().rename(columns = {'index':'STORE_ID','STORE_ID': 'TRAIN_COUNT'})
temp = temp.merge(pd.DataFrame(df_test['STORE_ID'].value_counts()).reset_index().rename(columns = {'index':'STORE_ID','STORE_ID': 'TEST_COUNT'}), on = 'STORE_ID', how = 'outer')
temp.head()

,STORE_ID,TRAIN_COUNT,TEST_COUNT
0,44,170730,41727
1,45,169795,41709
2,47,168077,41230
3,49,164636,40232
4,3,163865,39607


# Number of records for each ATRICLE_ID

In [10]:
temp = pd.DataFrame(df_train['ARTICLE_ID'].value_counts()).reset_index().rename(columns = {'index':'ARTICLE_ID','ARTICLE_ID': 'TRAIN_COUNT'})
temp = temp.merge(pd.DataFrame(df_test['ARTICLE_ID'].value_counts()).reset_index().rename(columns = {'index':'ARTICLE_ID','ARTICLE_ID': 'TEST_COUNT'}), on = 'ARTICLE_ID', how = 'outer')
temp[temp['TRAIN_COUNT'].isna()]

,ARTICLE_ID,TRAIN_COUNT,TEST_COUNT
3873,2123727,NaN,83.0
3874,2122676,NaN,66.0
3875,2123468,NaN,65.0
3876,2123775,NaN,64.0
3877,2123410,NaN,55.0
3878,2123750,NaN,50.0
3879,2122868,NaN,48.0
3880,1980754,NaN,29.0
3881,2120723,NaN,28.0
3882,2114752,NaN,25.0


**Есть ARTICLE_IDs который встречаются лишь в TEST_DF

# Create the DataFrame

In [11]:
df = df_train.append(df_test)
df

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION
0,2017-06-01,1,99197,3.0,0
1,2017-06-01,1,103520,2.0,0
2,2017-06-01,1,105574,11.0,0
3,2017-06-01,1,105575,9.0,0
4,2017-06-01,1,105577,2.0,0
...,...,...,...,...,...
1570963,2017-08-15,54,2089339,NaN,0
1570964,2017-08-15,54,2106464,NaN,1
1570965,2017-08-15,54,2110456,NaN,0
1570966,2017-08-15,54,2113914,NaN,1


## Attributes based on DAY_ID

In [12]:
df['MONTH']=df['DAY_ID'].dt.month
df['WEEK']=df['DAY_ID'].dt.week
df['DAY']=df['DAY_ID'].dt.day
df['WEEK_DAY']=df['DAY_ID'].dt.weekday # 0 - Monday, 4 - Friday
df['IS_WEEKEND']=[0 if x<5 else 1 for x in df['WEEK_DAY']]
df['DAY_OF_YEAR'] = df['DAY_ID'].dt.dayofyear
df

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152
1,2017-06-01,1,103520,2.0,0,6,22,1,3,0,152
2,2017-06-01,1,105574,11.0,0,6,22,1,3,0,152
3,2017-06-01,1,105575,9.0,0,6,22,1,3,0,152
4,2017-06-01,1,105577,2.0,0,6,22,1,3,0,152
...,...,...,...,...,...,...,...,...,...,...,...
1570963,2017-08-15,54,2089339,NaN,0,8,33,15,1,0,227
1570964,2017-08-15,54,2106464,NaN,1,8,33,15,1,0,227
1570965,2017-08-15,54,2110456,NaN,0,8,33,15,1,0,227
1570966,2017-08-15,54,2113914,NaN,1,8,33,15,1,0,227


In [13]:
df_holidays['IS_HOLIDAY'] = 1
df = df.merge(df_holidays, on = 'DAY_ID', how = 'left')

df['IS_HOLIDAY'] = df['IS_HOLIDAY'].fillna(0)
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0
1,2017-06-01,1,103520,2.0,0,6,22,1,3,0,152,0.0
2,2017-06-01,1,105574,11.0,0,6,22,1,3,0,152,0.0
3,2017-06-01,1,105575,9.0,0,6,22,1,3,0,152,0.0
4,2017-06-01,1,105577,2.0,0,6,22,1,3,0,152,0.0


### Attribute for the ARTICLE + STORE

In [14]:
df['FIRST_SALE_DAY'] = df['DAY_ID'].dt.dayofyear 

df['FIRST_SALE_DAY'] = df.groupby('ARTICLE_ID')['FIRST_SALE_DAY'].transform('min').astype('int16')
df['PREV_DAYS_ON_SALE'] = df['DAY_OF_YEAR'] - df['FIRST_SALE_DAY']
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0
1,2017-06-01,1,103520,2.0,0,6,22,1,3,0,152,0.0,152,0
2,2017-06-01,1,105574,11.0,0,6,22,1,3,0,152,0.0,152,0
3,2017-06-01,1,105575,9.0,0,6,22,1,3,0,152,0.0,152,0
4,2017-06-01,1,105577,2.0,0,6,22,1,3,0,152,0.0,152,0


### Attributes for DAY OF WEEK and STORE

In [15]:
temp = df.groupby(['STORE_ID','WEEK_DAY']).agg({'SALES':'sum'}).reset_index()
temp = temp.merge(df.groupby(['STORE_ID']).agg({'SALES':'sum'}).reset_index(), on=['STORE_ID'], how='left')
temp.columns = ['STORE_ID','WEEK_DAY','STORE_DAY_SALES', 'STORE_TOTAL_SALES']
temp['DAY_QUALITY'] = temp['STORE_DAY_SALES']/temp['STORE_TOTAL_SALES']
temp = temp[['STORE_ID','WEEK_DAY','DAY_QUALITY']]
temp.head()

,STORE_ID,WEEK_DAY,DAY_QUALITY
0,1,0,0.156606
1,1,1,0.139085
2,1,2,0.166687
3,1,3,0.143217
4,1,4,0.169450


In [16]:
df = df.merge(temp, on = ['STORE_ID', 'WEEK_DAY'], how = 'inner')
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0,0.143217
1,2017-06-01,1,103520,2.0,0,6,22,1,3,0,152,0.0,152,0,0.143217
2,2017-06-01,1,105574,11.0,0,6,22,1,3,0,152,0.0,152,0,0.143217
3,2017-06-01,1,105575,9.0,0,6,22,1,3,0,152,0.0,152,0,0.143217
4,2017-06-01,1,105577,2.0,0,6,22,1,3,0,152,0.0,152,0,0.143217


In [17]:
temp = pd.DataFrame(df['MONTH'].unique(), columns = ['MONTH'])
#temp.columns = ['MONTH', 'DAYS_CNT']
temp['DAYS_CNT'] = temp['MONTH'].apply(lambda x: calendar.monthrange(2017,x)[1])
temp

,MONTH,DAYS_CNT
0,6,30
1,7,31
2,8,31


In [1]:
df = df.merge(temp, on = 'MONTH', how = 'inner')
df['DAY_QUALITY'] = df['DAY_QUALITY']*df['DAYS_CNT']
df.drop('DAYS_CNT', axis = 1, inplace=True)

NameError: name 'df' is not defined

## Attributes for ARTICLE

In [19]:
df_article['ARTICLE_GROUP'].unique()

array(['GROCERY I', 'CLEANING', 'BREAD/BAKERY', 'DELI', 'POULTRY', 'EGGS',
       'PERSONAL CARE', 'LINGERIE', 'BEVERAGES', 'AUTOMOTIVE', 'DAIRY',
       'GROCERY II', 'MEATS', 'FROZEN FOODS', 'HOME APPLIANCES',
       'SEAFOOD', 'PREPARED FOODS', 'LIQUOR,WINE,BEER', 'BEAUTY',
       'HARDWARE', 'LAWN AND GARDEN', 'PRODUCE', 'HOME AND KITCHEN II',
       'HOME AND KITCHEN I', 'MAGAZINES', 'HOME CARE', 'PET SUPPLIES',
       'BABY CARE', 'SCHOOL AND OFFICE SUPPLIES',
       'PLAYERS AND ELECTRONICS', 'CELEBRATION', 'LADIESWEAR', 'BOOKS'],
      dtype=object)

In [20]:
df_article.groupby(['ARTICLE_GROUP', 'ARTICLE_CLASS']).count().reset_index()

,ARTICLE_GROUP,ARTICLE_CLASS,ARTICLE_ID
0,AUTOMOTIVE,6806,5
1,AUTOMOTIVE,6808,2
2,AUTOMOTIVE,6810,6
3,AUTOMOTIVE,6824,2
4,AUTOMOTIVE,6848,5
5,BABY CARE,6022,1
6,BEAUTY,4210,1
7,BEAUTY,4212,1
8,BEAUTY,4214,2
9,BEAUTY,4222,2


In [21]:
df = df.merge(df_article, how = 'inner', on = 'ARTICLE_ID')
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1067
1,2017-06-08,1,99197,1.0,0,6,23,8,3,0,159,0.0,152,7,4.296499,GROCERY I,1067
2,2017-06-15,1,99197,2.0,0,6,24,15,3,0,166,0.0,152,14,4.296499,GROCERY I,1067
3,2017-06-29,1,99197,1.0,0,6,26,29,3,0,180,0.0,152,28,4.296499,GROCERY I,1067
4,2017-06-08,3,99197,3.0,0,6,23,8,3,0,159,0.0,152,7,3.318019,GROCERY I,1067


In [22]:
temp = df.groupby(['ARTICLE_GROUP', 'ARTICLE_CLASS','STORE_ID', 'MONTH', 'DAY']).agg({'SALES': 'sum'}).reset_index()
temp.head()

,ARTICLE_GROUP,ARTICLE_CLASS,STORE_ID,MONTH,DAY,SALES
0,AUTOMOTIVE,6806,1,6,1,4.0
1,AUTOMOTIVE,6806,1,6,2,7.0
2,AUTOMOTIVE,6806,1,6,3,3.0
3,AUTOMOTIVE,6806,1,6,5,2.0
4,AUTOMOTIVE,6806,1,6,7,3.0


In [23]:
temp = temp[temp['DAY']<=15].groupby(['ARTICLE_GROUP', 'ARTICLE_CLASS','STORE_ID', 'MONTH'])['SALES']\
.sum().reset_index().rename(columns = {'SALES': 'FIRST_PART_MONTH_SALES'})\
.merge(temp[temp['DAY']>15].groupby(['ARTICLE_GROUP', 'ARTICLE_CLASS','STORE_ID', 'MONTH'])['SALES']\
.sum().reset_index().rename(columns = {'SALES': 'SECOND_PART_MONTH_SALES'}), on = ['ARTICLE_GROUP'\
,'ARTICLE_CLASS','STORE_ID', 'MONTH'], how = 'inner') 

temp['NEXT_MONTH'] = temp['MONTH'].apply(lambda x: (x + 1) %12)
temp.head()

,ARTICLE_GROUP,ARTICLE_CLASS,STORE_ID,MONTH,FIRST_PART_MONTH_SALES,SECOND_PART_MONTH_SALES,NEXT_MONTH
0,AUTOMOTIVE,6806,1,6,25.0,25.0,7
1,AUTOMOTIVE,6806,1,7,26.0,39.0,8
2,AUTOMOTIVE,6806,2,6,16.0,36.0,7
3,AUTOMOTIVE,6806,2,7,22.0,38.0,8
4,AUTOMOTIVE,6806,3,6,68.0,81.0,7


In [24]:
temp1 = temp.groupby(['ARTICLE_GROUP', 'MONTH', 'NEXT_MONTH'])\
.agg({'FIRST_PART_MONTH_SALES': sum, 'SECOND_PART_MONTH_SALES': sum}).reset_index()\
.rename(columns = {'FIRST_PART_MONTH_SALES': 'FIRST_PART_MONTH_SALES_ARTICLE_GROUP', 
                   'SECOND_PART_MONTH_SALES': 'SECOND_PART_MONTH_SALES_ARTICLE_GROUP'})
temp1

,ARTICLE_GROUP,MONTH,NEXT_MONTH,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP
0,AUTOMOTIVE,6,7,6158.000,5793.000
1,AUTOMOTIVE,7,8,5974.000,6192.000
2,BABY CARE,6,7,144.000,135.000
3,BABY CARE,7,8,191.000,150.000
4,BEAUTY,6,7,4253.000,4522.000
5,BEAUTY,7,8,4933.000,4661.000
6,BEVERAGES,6,7,3090944.000,2708845.000
7,BEVERAGES,7,8,3015565.000,3188421.000
8,BOOKS,6,7,33.000,19.000
9,BOOKS,7,8,16.000,15.000


In [25]:
df = df.merge(temp1.drop('MONTH', axis = 1), left_on = ['MONTH', 'ARTICLE_GROUP'], \
              right_on = ['NEXT_MONTH', 'ARTICLE_GROUP'], how = 'left')
df.drop('NEXT_MONTH', axis = 1, inplace = True)
df['FIRST_PART_MONTH_SALES_ARTICLE_GROUP'] = df['FIRST_PART_MONTH_SALES_ARTICLE_GROUP'].fillna(0)
df['SECOND_PART_MONTH_SALES_ARTICLE_GROUP'] = df['SECOND_PART_MONTH_SALES_ARTICLE_GROUP'].fillna(0)
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1067,0.0,0.0
1,2017-06-08,1,99197,1.0,0,6,23,8,3,0,159,0.0,152,7,4.296499,GROCERY I,1067,0.0,0.0
2,2017-06-15,1,99197,2.0,0,6,24,15,3,0,166,0.0,152,14,4.296499,GROCERY I,1067,0.0,0.0
3,2017-06-29,1,99197,1.0,0,6,26,29,3,0,180,0.0,152,28,4.296499,GROCERY I,1067,0.0,0.0
4,2017-06-08,3,99197,3.0,0,6,23,8,3,0,159,0.0,152,7,3.318019,GROCERY I,1067,0.0,0.0


In [26]:
temp1 = temp.groupby(['ARTICLE_CLASS', 'MONTH', 'NEXT_MONTH'])\
.agg({'FIRST_PART_MONTH_SALES': sum, 'SECOND_PART_MONTH_SALES': sum}).reset_index()\
.rename(columns = {'FIRST_PART_MONTH_SALES': 'FIRST_PART_MONTH_SALES_ARTICLE_CLASS', 
                   'SECOND_PART_MONTH_SALES': 'SECOND_PART_MONTH_SALES_ARTICLE_CLASS'})
temp1

,ARTICLE_CLASS,MONTH,NEXT_MONTH,FIRST_PART_MONTH_SALES_ARTICLE_CLASS,SECOND_PART_MONTH_SALES_ARTICLE_CLASS
0,1002,6,7,40443.864,39441.707
1,1002,7,8,43344.624,49203.057
2,1003,6,7,3639.000,3604.000
3,1003,7,8,3874.000,3986.000
4,1004,6,7,179379.000,167152.000
...,...,...,...,...,...
629,7016,7,8,1032.000,928.000
630,7034,6,7,3077.000,3083.000
631,7034,7,8,3219.000,3090.000
632,7780,6,7,9895.000,9438.000


In [27]:
df = df.merge(temp1.drop('MONTH', axis = 1), left_on = ['MONTH', 'ARTICLE_CLASS'], \
              right_on = ['NEXT_MONTH', 'ARTICLE_CLASS'], how = 'left')
df.drop('NEXT_MONTH', axis = 1, inplace = True)
df['FIRST_PART_MONTH_SALES_ARTICLE_CLASS'] = df['FIRST_PART_MONTH_SALES_ARTICLE_CLASS'].fillna(0)
df['SECOND_PART_MONTH_SALES_ARTICLE_CLASS'] = df['SECOND_PART_MONTH_SALES_ARTICLE_CLASS'].fillna(0)
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP,FIRST_PART_MONTH_SALES_ARTICLE_CLASS,SECOND_PART_MONTH_SALES_ARTICLE_CLASS
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0
1,2017-06-08,1,99197,1.0,0,6,23,8,3,0,159,0.0,152,7,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0
2,2017-06-15,1,99197,2.0,0,6,24,15,3,0,166,0.0,152,14,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0
3,2017-06-29,1,99197,1.0,0,6,26,29,3,0,180,0.0,152,28,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0
4,2017-06-08,3,99197,3.0,0,6,23,8,3,0,159,0.0,152,7,3.318019,GROCERY I,1067,0.0,0.0,0.0,0.0


In [28]:
temp1 = temp.groupby(['STORE_ID', 'MONTH', 'NEXT_MONTH'])\
.agg({'FIRST_PART_MONTH_SALES': sum, 'SECOND_PART_MONTH_SALES': sum}).reset_index()\
.rename(columns = {'FIRST_PART_MONTH_SALES': 'FIRST_PART_MONTH_SALES_STORE_ID', 
                   'SECOND_PART_MONTH_SALES': 'SECOND_PART_MONTH_SALES_STORE_ID'})
temp1

,STORE_ID,MONTH,NEXT_MONTH,FIRST_PART_MONTH_SALES_STORE_ID,SECOND_PART_MONTH_SALES_STORE_ID
0,1,6,7,170702.181,165332.015
1,1,7,8,161125.908,162764.862
2,2,6,7,238485.982,232237.358
3,2,7,8,235763.333,238481.532
4,3,6,7,598389.639,552940.133
5,3,7,8,574289.298,585068.261
6,4,6,7,207244.395,195522.839
7,4,7,8,197536.871,205490.549
8,5,6,7,156999.400,148372.761
9,5,7,8,157881.508,167728.427


In [29]:
df = df.merge(temp1.drop('MONTH', axis = 1), left_on = ['MONTH', 'STORE_ID'], right_on = ['NEXT_MONTH', 'STORE_ID'], \
              how = 'left')
df.drop('NEXT_MONTH', axis = 1, inplace = True)
df['FIRST_PART_MONTH_SALES_STORE_ID'] = df['FIRST_PART_MONTH_SALES_STORE_ID'].fillna(0)
df['SECOND_PART_MONTH_SALES_STORE_ID'] = df['SECOND_PART_MONTH_SALES_STORE_ID'].fillna(0)
df.head()

,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP,FIRST_PART_MONTH_SALES_ARTICLE_CLASS,SECOND_PART_MONTH_SALES_ARTICLE_CLASS,FIRST_PART_MONTH_SALES_STORE_ID,SECOND_PART_MONTH_SALES_STORE_ID
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0
1,2017-06-08,1,99197,1.0,0,6,23,8,3,0,159,0.0,152,7,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0
2,2017-06-15,1,99197,2.0,0,6,24,15,3,0,166,0.0,152,14,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0
3,2017-06-29,1,99197,1.0,0,6,26,29,3,0,180,0.0,152,28,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0
4,2017-06-08,3,99197,3.0,0,6,23,8,3,0,159,0.0,152,7,3.318019,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0


## Attributes for STORE

In [30]:
df = df.merge(df_store, on = 'STORE_ID', how = 'inner')
print(df.shape)
df.head()

(8019163, 27)


,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP,FIRST_PART_MONTH_SALES_ARTICLE_CLASS,SECOND_PART_MONTH_SALES_ARTICLE_CLASS,FIRST_PART_MONTH_SALES_STORE_ID,SECOND_PART_MONTH_SALES_STORE_ID,CITY_ID,REGION_ID,TYPE_ID,CLUSTER_ID
0,2017-06-01,1,99197,3.0,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13
1,2017-06-08,1,99197,1.0,0,6,23,8,3,0,159,0.0,152,7,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13
2,2017-06-15,1,99197,2.0,0,6,24,15,3,0,166,0.0,152,14,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13
3,2017-06-29,1,99197,1.0,0,6,26,29,3,0,180,0.0,152,28,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13
4,2017-06-02,1,99197,2.0,0,6,22,2,4,0,153,0.0,152,1,5.083510,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13


In [31]:
df_sales_count

,DAY_ID,STORE_ID,SALES_COUNT
0,2017-06-01,1,1815
1,2017-06-01,2,1802
2,2017-06-01,3,3066
3,2017-06-01,4,1289
4,2017-06-01,5,1266
...,...,...,...
4099,2017-08-15,50,2804
4100,2017-08-15,51,1573
4101,2017-08-15,52,2255
4102,2017-08-15,53,932


In [32]:
df = df.merge(df_sales_count, on = ['DAY_ID', 'STORE_ID'])
print(df.shape)
df

(8019163, 28)


,DAY_ID,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP,FIRST_PART_MONTH_SALES_ARTICLE_CLASS,SECOND_PART_MONTH_SALES_ARTICLE_CLASS,FIRST_PART_MONTH_SALES_STORE_ID,SECOND_PART_MONTH_SALES_STORE_ID,CITY_ID,REGION_ID,TYPE_ID,CLUSTER_ID,SALES_COUNT
0,2017-06-01,1,99197,3.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13,1815
1,2017-06-01,1,103520,2.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1028,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13,1815
2,2017-06-01,1,105574,11.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1045,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13,1815
3,2017-06-01,1,105575,9.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1045,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13,1815
4,2017-06-01,1,105577,2.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,GROCERY I,1045,0.0,0.0,0.0,0.0,0.0,0.0,0,0,D,13,1815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019158,2017-06-07,35,1445215,4.000,0,6,23,7,2,0,158,0.0,152,6,2.965928,FROZEN FOODS,2246,0.0,0.0,0.0,0.0,0.0,0.0,14,8,C,3,605
8019159,2017-06-07,35,584032,16.714,0,6,23,7,2,0,158,0.0,152,6,2.965928,MEATS,2302,0.0,0.0,0.0,0.0,0.0,0.0,14,8,C,3,605
8019160,2017-06-07,35,1970094,1.000,0,6,23,7,2,0,158,0.0,152,6,2.965928,AUTOMOTIVE,6848,0.0,0.0,0.0,0.0,0.0,0.0,14,8,C,3,605
8019161,2017-06-07,35,1935601,1.000,0,6,23,7,2,0,158,0.0,152,6,2.965928,GROCERY I,1004,0.0,0.0,0.0,0.0,0.0,0.0,14,8,C,3,605


In [33]:
X_train = df[df['SALES'].notna()].drop('DAY_ID', axis = 1)

In [34]:
# define ordinal encoding
encoder = OrdinalEncoder(handle_unknown='value',handle_missing='value')
# transform data
X_train = encoder.fit_transform(X_train, cols = ['ARTICLE_GROUP'])
X_train

,STORE_ID,ARTICLE_ID,SALES,IS_ACTION,MONTH,WEEK,DAY,WEEK_DAY,IS_WEEKEND,DAY_OF_YEAR,IS_HOLIDAY,FIRST_SALE_DAY,PREV_DAYS_ON_SALE,DAY_QUALITY,ARTICLE_GROUP,ARTICLE_CLASS,FIRST_PART_MONTH_SALES_ARTICLE_GROUP,SECOND_PART_MONTH_SALES_ARTICLE_GROUP,FIRST_PART_MONTH_SALES_ARTICLE_CLASS,SECOND_PART_MONTH_SALES_ARTICLE_CLASS,FIRST_PART_MONTH_SALES_STORE_ID,SECOND_PART_MONTH_SALES_STORE_ID,CITY_ID,REGION_ID,TYPE_ID,CLUSTER_ID,SALES_COUNT
0,1,99197,3.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,1,1067,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,13,1815
1,1,103520,2.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,1,1028,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,13,1815
2,1,105574,11.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,1,1045,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,13,1815
3,1,105575,9.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,1,1045,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,13,1815
4,1,105577,2.000,0,6,22,1,3,0,152,0.0,152,0,4.296499,1,1045,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,13,1815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8019158,35,1445215,4.000,0,6,23,7,2,0,158,0.0,152,6,2.965928,16,2246,0.0,0.0,0.0,0.0,0.0,0.0,14,8,4,3,605
8019159,35,584032,16.714,0,6,23,7,2,0,158,0.0,152,6,2.965928,12,2302,0.0,0.0,0.0,0.0,0.0,0.0,14,8,4,3,605
8019160,35,1970094,1.000,0,6,23,7,2,0,158,0.0,152,6,2.965928,14,6848,0.0,0.0,0.0,0.0,0.0,0.0,14,8,4,3,605
8019161,35,1935601,1.000,0,6,23,7,2,0,158,0.0,152,6,2.965928,1,1004,0.0,0.0,0.0,0.0,0.0,0.0,14,8,4,3,605


In [35]:
X_val = X_train[(X_train['MONTH']==7)&(X_train['DAY']>23)]
X_train = X_train[((X_train['MONTH']==7)&(X_train['DAY']<=23))|(X_train['MONTH']!=7)]

y_train = X_train['SALES']
y_val = X_val['SALES']

X_train.drop('SALES', axis = 1, inplace = True) 
X_val.drop('SALES', axis = 1, inplace = True) 
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)

print('X_val.shape: ', X_val.shape)
print('y_val.shape: ', y_val.shape)

X_train.shape:  (5611922, 26)
y_train.shape:  (5611922,)
X_val.shape:  (836273, 26)
y_val.shape:  (836273,)


In [36]:
X_test = df[df['SALES'].isna()].drop('DAY_ID', axis = 1)
X_test = encoder.transform(X_test)
X_test.drop('SALES', axis = 1, inplace = True)
print('X_test.shape: ', X_test.shape)

X_test.shape:  (1570968, 26)


In [37]:
del df

In [39]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 1000
}

In [40]:
gbm = lgbm.LGBMRegressor(**hyper_params)
gbm.fit(X_train, y_train)

[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.252277 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.


LGBMRegressor(bagging_fraction=0.7, bagging_freq=10, boosting_type='gbdt',
              class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
              importance_type='split', learning_rate=0.005, max_bin=512,
              max_depth=8, metric=['l1', 'l2'], min_child_samples=20,
              min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
              n_jobs=-1, num_iterations=1000, num_leaves=128,
              objective='regression', random_state=None, reg_alpha=0.0,
              reg_lambda=0.0, silent='warn', subsample=1.0,
              subsample_for_bin=200000, subsample_freq=0, task='train',
              verbose=0)

In [41]:
y_pred_train = gbm.predict(X_train, num_iteration=gbm.best_iteration_)
y_pred_val = gbm.predict(X_val, num_iteration=gbm.best_iteration_)

In [48]:
print('Training set rmse: {:.4f}'.format(rmse(y_train, y_pred_train)))
print('Test set rmse: {:.4f}'.format(rmse(y_val, y_pred_val)))

Training set rmse: 21.8658
Test set rmse: 23.3215


In [49]:
df_submission['SALES'] = gbm.predict(X_test, num_iteration=gbm.best_iteration_).clip(0,20)
df_submission

,DAY_ID,STORE_ID,ARTICLE_ID,SALES
0,2017-08-01,1,103520,1.745708
1,2017-08-01,1,103665,5.276318
2,2017-08-01,1,105574,5.276318
3,2017-08-01,1,105575,3.159241
4,2017-08-01,1,105693,2.616565
...,...,...,...,...
1570963,2017-08-15,54,2089339,3.388400
1570964,2017-08-15,54,2106464,1.367103
1570965,2017-08-15,54,2110456,10.593287
1570966,2017-08-15,54,2113914,2.337943


In [50]:
df_submission.to_csv('regression_df_submission_predicted.csv', index=False)